# Compute the Frost Days Climate Index

This notebook provides an example of climate data analysis performed in parallel with Dask.

Let TN be the Daily Minimum Near-Surface (usually, 2 meter) Air Temperature. The Frost Days index is the number of days where

\begin{equation*} 
$TN < 0°
\end{equation*}


The notebook will exploit:

  - the **intake** catalog to search and discover the CMIP6 datasets available in the ENES Data Space archive
  - **Dask** and **xarray** to compute the **Frost Days** climate index in parallel
  - **hvPlot**, **Cartopy** and **Matplotlib** to plot the analysis results

Make sure to select the **Python 3 (ipykernel)** Notebook Kernel.

In [ ]:
import os
import dask
import intake
import xarray as xr
import numpy as np
import ipywidgets as ipw
import hvplot.xarray
import hvplot.pandas
import panel as pn
import pandas as pd
import panel.widgets as pnw
from os.path import expanduser
home = expanduser("~")
from dask.distributed import Client, LocalCluster
worker_kwargs = {'local_directory': '/home/jovyan'}

### Search & Discover data with **intake-esm**

Open the ESM collection file

In [ ]:
esm_file = home+"/data/CMIP6_ESM_colletion_file.json"
col = intake.open_esm_datastore(esm_file)
col.df.head()

Select the desired *experiment*, *model* and *time resolution*.

For example, we are interested in:
  - `ssp585` experiment from the `CMCC-ESM2` model
  - `tasmin` variable at `daily` frequency
  - `r1i1p1f1` ensemble member

In [ ]:
experiments = col.unique().loc[["experiment_id"]][0]

In [ ]:
experiment = widgets.Dropdown(
    options=[e for e in experiments],
    value='ssp585',
    description='Experiment:',
)

display(experiment)

In [ ]:
sources = col.df.loc[col.df['experiment_id'] == experiment.value]["source_id"].unique()

In [ ]:
source = widgets.Dropdown(
    options=[s for s in sources],
    value='CMCC-ESM2',
    description='Source ID:',
)

display(source)

In [ ]:
table_id = widgets.RadioButtons(
    options=['6hrLev','day', 'Amon'],
    value='day',
    description='table ID',
    disabled=False
)
display(table_id)

Perform the query against the data catalogue

In [ ]:
import intake
esm_file = home+"/data/CMIP6_ESM_colletion_file.json"
col = intake.open_esm_datastore(esm_file)

query = dict( 
    experiment_id=experiment.value,
    source_id=source.value,
    variable_id="tasmin",
    table_id=table_id.value,
    member_id="r1i1p1f1"
)
cat = col.search(**query)
paths = cat.df['path']#.values
cat.df

### Deploy a **Dask** Local cluster

- Create local Scheduler and Workers: this creates a “cluster” of a scheduler and workers running on the local machine
- Connect to and submit computation to the Dask cluster

In [ ]:
cluster = LocalCluster(n_workers=2, threads_per_worker=1, **worker_kwargs)
client = Client(cluster)
client

### Start the computation

Inspect the data structure

In [ ]:
ds = xr.open_mfdataset(paths, combine='nested', concat_dim="time", parallel=True, engine='netcdf4')
ds["tasmin"]

Plot the Daily Minimum Near-Surface Air Temperature for each timestep in the input datasets

In [ ]:
ds["tasmin"].interactive.sel(time=pnw.DiscreteSlider).plot()

Define the EU bounding box

In [ ]:
# EUROPE
north= 70.94985882321207
south= 35.049860308608224
east= 40
west= -24.950139509199722

Count the number of frost days on yearly basis over the EU domain:

\begin{equation*} 
    FrostDays = \{day| TN(day) < 273.15 \}
\end{equation*} 

Note that the temperature unit is K.

In [ ]:
def calc_frost_days(tasmin_ds, threshold):

    total = (tasmin_ds.sel(lat=slice(south,north), lon=slice(west,east)) < threshold).groupby('time.year').sum()
    return total
 
frost_days = calc_frost_days(ds['tasmin'], 273.15)
result = frost_days.compute()
frost_days

Use the Player widget from the Panel library to plot the result in an interactive way

In [ ]:
time = pnw.Player(name='year', start=0, end=len(result.year)-1, loop_policy='loop', interval=1000)

result.interactive.isel(year=time).plot()

Export the result to a NetCDF file

In [ ]:
result.to_netcdf(path=home+"/work/frost_days.nc", format="NETCDF4",engine="netcdf4")

Check the NetCDF header information through the **ncdump** command-line utility

In [ ]:
! ncdump -h '/home/jovyan/work/frost_days.nc'

Create an animated image with **Cartopy** and **Matplotlib** showing the yearly Frost Days in Europe until 2100.

In [ ]:
%matplotlib inline
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.util import add_cyclic_point
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import imageio
import ipywidgets as widgets

images = []
img=home+'/work/yearly_frost_days_'

start_year=np.min(result.year.values)
end_year=np.max(result.year.values)
#levStep = (np.nanmax(result.values)-np.nanmin(result.values))/20
#clevs = np.arange(np.nanmin(result.values),np.nanmax(result.values)+levStep,levStep)

levStep = (300-0)/20
clevs = np.arange(0,300+levStep,levStep)

for i in result.year.values:
    
    lat = result.sel(year=i).lat.values
    lon = result.sel(year=i).lon.values
    var = result.sel(year=i).values
    var = np.reshape(var, (len(lat), len(lon)))

    fig = plt.figure(figsize=(15, 6), dpi=100)

    #Add Geo axes to the figure with the specified projection (PlateCarree)
    projection = ccrs.PlateCarree()
    ax = plt.axes(projection=projection)

    #Draw coastline and gridlines
    ax.coastlines()

    gl = ax.gridlines(crs=projection, draw_labels=True, linewidth=1, color='black', alpha=0.9, linestyle=':')
    gl.xlabels_top = False
    gl.ylabels_right = False
    
    #Wraparound points in longitude
    var_cyclic, lon_cyclic = add_cyclic_point(var, coord=np.asarray(lon))
    x, y = np.meshgrid(lon_cyclic,lat)

    #Set filled contour plot
    cnplot = ax.contourf(x, y, var_cyclic, clevs, transform=projection,cmap=plt.cm.Oranges)
    plt.colorbar(cnplot,ax=ax)
    
    plt.title('Frost Days (year ' + str(i)+')')
    plt.savefig(img+str(i)+'.png', dpi=fig.dpi, bbox_inches='tight')
    plt.close()

    images.append(imageio.imread(img+str(i)+'.png')) 
    try:
        os.remove(img+str(i)+'.png')
    except:
        pass
             
args = { 'duration': 0.5 }
imageio.mimsave(home+'/work/Frost Days_{}-{}.gif'.format(start_year,end_year), images, 'gif', **args)

In [ ]:
with open(home+'/work/Frost Days_{}-{}.gif'.format(start_year,end_year), "rb") as file:
    image = file.read()

widgets.Image(
    value=image,
    format='gif'
)

### Shutdown the **Dask** Local cluster

When you’re done using it, you can shutdown the cluster: this will cleanly close all the connected dask workers, as well as the scheduler. 

In [ ]:
client.shutdown()

In [ ]:
client